# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Monitoring Pipeline</span>


## 🗒️ This notebook is divided into the following sections:

1. Fetch the model and corresponding feature view to read logs.
2. Use the Logs to monitor the model for drifts.

In [2]:
import hopsworks
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone

## Connect to Hopsworks
Establish connection to the Hopsworks feture store.

In [3]:
project = hopsworks.login()

fs = project.get_feature_store()

mr = project.get_model_registry()

ms = project.get_model_serving()

2025-11-21 14:06:56,064 INFO: Python Engine initialized.

Logged in to project, explore it here https://hopsworks.ai.local/p/120


## Fetch the model to be monitored using the deployment and fetch the model from it

In [ ]:
deployment = ms.get_deployment("fraudonlinemodeldeployment")

retrieved_model = mr.get_model(
    name=deployment.model_name,
    version=deployment.model_version,
)

# Get the feature view used to train the model
feature_view = retrieved_model.get_feature_view() 

## Read the required logs from the feature view.

In the example below, we read logging written in the last year.

In [ ]:
# Read the required logs from the feature view
logs = feature_view.read_log(start_time=datetime.now(timezone.utc) - timedelta(days=1), end_time=datetime.now(timezone.utc))
logs.head()

## Vizualise distribution of model inputs features and model training dataset features

In [ ]:
model_training_dataset_version = retrieved_model.get_training_dataset_provenance().accessible[0].version

In [ ]:
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(model_training_dataset_version)

In [ ]:
# Create the figure
fig = go.Figure()

# Add first histogram
fig.add_trace(go.Histogram(
    x=X_train['amount'],
    name='Training Data',
    opacity=0.6
))

# Add second histogram
fig.add_trace(go.Histogram(
    x=logs['amount'],
    name='Model Input',
    opacity=0.6
))

# Overlay histograms
fig.update_layout(
    barmode='overlay',
    title='Distribution Comparison - age_at_transaction',
    xaxis_title='amount',
    yaxis_title='Count'
)

fig.show()

## Integrating with other party libaries
One you have the logs stored you can read it up to integrate it with other model monitoring softwares like for example nannyml.

Here you an example of using nannyml's univarite drift detection using the logged features

In [ ]:
! pip install nannyml --quiet

In [ ]:
import nannyml as nml

In [ ]:
feature_column_names = ["amount", "label_encoder_country_"]

In [ ]:
univariate_calculator = nml.UnivariateDriftCalculator(
    column_names=feature_column_names,
    chunk_size=10
)

In [ ]:
univariate_calculator.fit(X_train)

In [ ]:
univariate_drift = univariate_calculator.calculate(logs)

In [ ]:
univariate_drift.plot()